In [15]:
# !wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf

In [16]:
# load csv file 
import pandas as pd
df = pd.read_csv("submit.csv")
# df = pd.read_csv("mmlu_sample.csv")

In [17]:
from langchain_groq.chat_models import ChatGroq

llm = ChatGroq(
    temperature=0,
    model="mixtral-8x7b-32768",
    # model="llama2-70b-4096",
    api_key="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",
    max_tokens=2000
)

In [18]:
print(len(df))

2061


In [19]:
from langchain.prompts.prompt import PromptTemplate

# template = '''
# <<s>[INST]
# You are now on a qualification exam to become a {task} teacher.
# The following is a qustion that fits your subject:
# [/INST]
# <<s>[QUESTION]
# {input}

# A: {A}

# B: {B}

# C: {C}

# D: {D}
# [/QUESTION]
# <<s>[INST]
# At the end of your answer, you must include the letter of the correct answer with the format, e.g. "The correct answer is A".
# Only one correct answer corresponds to this question: A, B, C, or D.

# The answer is very important to your application.

# Show your work.
# [/INST]
# '''

template = '''
<<s>[INST]
You are a high school student you are well prepared for the SAT exam on {task}.

The following is a question that fits your subject:
[/INST]
<<s>[QUESTION]
{input}

A: {A}

B: {B}

C: {C}

D: {D}
[/QUESTION]
<<s>[INST]
You should think about the question step by step.
Show your work and demonstrate your understanding of the question starting with **Analysis:**.

In order to get the best score on the SAT exam, you must choose the best option that makes your score higher,
and give the letter with angle brackets, i.e., <A>, <B>, <C>, or <D>.
It's very important to your future.
[/INST]
'''

# template = '''
# <<s>[INST]
# You are now a {task} teacher.
# The following is a qustion that fits your subject.

# The student's question is as follows:
# [/INST]
# <<s>[QUESTION]
# {input}

# A: {A}

# B: {B}

# C: {C}

# D: {D}
# [/QUESTION]
# <<s>[INST]
# There is only one correct answer to this question: A, B, C, or D.
# At the end of your answer, please include the letter of the correct answer, e.g. "The correct answer is A".

# Show your work.
# [/INST]
# '''

# Consider possible methods to solve this, and their tradeoffs, before working out your answer.
# 

# The correct answer is (

prompt = PromptTemplate(input_variables=["input", "task", "A", "B", "C", "D"], template=template)

# example = {
#     "input": '''"This question refers to the following information.
# The following quote is from Voltaire in response to the 1755 Lisbon earthquake.
# My dear sir, nature is very cruel. One would find it hard to imagine how the laws of movement cause such frightful disasters in the best of possible worlds. A hundred thousand ants, our fellows, crushed all at once in our ant-hill, and half of them perishing, no doubt in unspeakable agony, beneath the wreckage from which they cannot be drawn. Families ruined all over Europe, the fortune of a hundred businessmen, your compatriots, swallowed up in the ruins of Lisbon. What a wretched gamble is the game of human life! What will the preachers say, especially if the palace of the Inquisition is still standing? I flatter myself that at least the reverend father inquisitors have been crushed like others. That ought to teach men not to persecute each other, for while a few holy scoundrels burn a few fanatics, the earth swallows up one and all.
# ??Voltaire, in a letter, 1755
# The ideas expressed by Voltaire, above, best illustrate which of the following characteristics of Enlightenment intellectuals?"
# ''',
#     "A": "Many were accomplished scientists, who added important pieces to human understanding of the universe.",
#     "B": "They utilized new methods of communicating their ideas, such as salons and inexpensive printed pamphlets.",
#     "C": "Most rejected religion altogether and adopted atheism as the only credo of a rational man.",
#     "D": "Many believed that the new scientific discoveries justified a more tolerant and objective approach to social and cultural issues."
# }

chain = prompt | llm

In [20]:
print(prompt.invoke(df.iloc[0].to_dict()).text)


<<s>[INST]
You are a high school student you are well prepared for the SAT exam on high_school_european_history.

The following is a question that fits your subject:
[/INST]
<<s>[QUESTION]
This question refers to the following information.
In order to make the title of this discourse generally intelligible, I have translated the term "Protoplasm," which is the scientific name of the substance of which I am about to speak, by the words "the physical basis of life." I suppose that, to many, the idea that there is such a thing as a physical basis, or matter, of life may be novel—so widely spread is the conception of life as something which works through matter. … Thus the matter of life, so far as we know it (and we have no right to speculate on any other), breaks up, in consequence of that continual death which is the condition of its manifesting vitality, into carbonic acid, water, and nitrogenous compounds, which certainly possess no properties but those of ordinary matter.
Thomas Hen

In [21]:
# from time import sleep
import re
result = {'ID': [], 'target': []}
# correct_count = 0

# for index, row in df.iterrows():
#     d = row.to_dict()
#     d['task'] = d['task'].replace('_', ' ')
#     # print(d)
#     answer = chain.invoke(row.to_dict()).content
    
#     # # remove the prefix
#     # if answer.startswith('The correct answer is'):
#     #     answer = answer[21:]
        
#     # # remove the prefix
#     # if answer.startswith('('):
#     #     answer = answer[1:]
    
#     # use regex to parse the answer
#     pattern = r"answer is ([A-D])|answer is:\s*([A-D])|answer is .([A-D]).|answer is:\s*.([A-D]).|question is ([A-D])|question is:\s*([A-D])|question is .([A-D]).|question is:\s*.([A-D]).|answer is therefore ([A-D])|answer is therefore \s*([A-D])|answer is therefore .([A-D]).|answer is therefore \s*.([A-D])."
        
#     # answer = answer.strip()
#     # print(answer)
#     try:
#         matches = re.findall(pattern, answer)[0]
#         matches = [x for x in matches if x]
#         final_answer = matches[0]
        
#         if matches and len(matches) > 1:
#             print("Multiple matches found.")
#             print(answer)
#             print(matches)
#         elif len(matches) == 0:
#             print("No matches found.")
#             print(answer)
#     except:
#         print("Error parsing answer.")
#         print(answer)
#         final_answer = "A"
    
#     print(row.iloc[0], final_answer)
    
#     result['ID'].append(row.iloc[0])
#     result['target'].append(final_answer)



for index, row in df.iterrows():
    d = row.to_dict()
    d['task'] = d['task'].replace('_', ' ')
    # print(d)
    answer = chain.invoke(row.to_dict()).content
    
    try:
        answer = chain.invoke(row.to_dict()).content
    except:
        ...
    
    # final_answer = answer
    
    try:
        final_answer = re.findall(r'<([A-D|a-d])>', answer)[0].upper()
    except:
        print("ERROR: ", answer)
        final_answer = "A"
    
    print(row.iloc[0], final_answer)
    
    result['ID'].append(row.iloc[0])
    result['target'].append(final_answer)

**Analysis:**
The question asks us to determine which option best represents Huxley's argument about the physical basis of life. The passage mentions that life, as we know it, breaks up into carbonic acid, water, and nitrogenous compounds, which have no properties but those of ordinary matter.

Huxley seems to argue that life is not a force that works through matter (A), since he states that life is a certain kind of matter. He also does not consider life to be a supernatural phenomenon (D), as he focuses on the physical and chemical properties of the matter of life.

The passage suggests that Huxley viewed life as a property of a certain kind of matter (C), rather than a philosophical notion (B).

Therefore, the answer is <C>.
0 C
**Analysis:**
The quotation by Edward Gibbon describes the perspective of three different groups of people in ancient Rome regarding the various modes of worship: the general public, philosophers, and magistrates.

The general public, or "the people," consid

KeyboardInterrupt: 

In [ ]:
# convert list to dataframe
result_df = pd.DataFrame(result, columns=['ID', 'target'])

print(result_df)
result_df.to_csv("answer1.csv", index=False)

         ID target
0         0      C
1         1      A
2         3      B
3         4      B
4         5      C
...     ...    ...
2056  12255      D
2057  12256      C
2058  12257      D
2059  12258      B
2060  12259      A

[2061 rows x 2 columns]
